In [ ]:
## Los recursos empleados en caso no se tengan:
]add RDatasets DataFrames MLJModels MLJDecisionTreeInterface DecisionTree StatsPlots

    Updating registry at `C:\Users\sophi\.julia\registries\General.toml`
   Resolving package versions...
   Installed libdecor_jll ───────────────── v0.2.2+0
   Installed x265_jll ───────────────────── v3.5.0+0
   Installed Libmount_jll ───────────────── v2.40.1+0
   Installed GR_jll ─────────────────────── v0.73.8+0
   Installed JpegTurbo_jll ──────────────── v3.0.4+0
   Installed libfdk_aac_jll ─────────────── v2.0.3+0
   Installed HypergeometricFunctions ────── v0.3.25
   Installed ScikitLearnBase ────────────── v0.5.0
   Installed OffsetArrays ───────────────── v1.14.1
   Installed LERC_jll ───────────────────── v4.0.0+0
   Installed Xorg_xkbcomp_jll ───────────── v1.4.6+0
   Installed Opus_jll ───────────────────── v1.3.3+0
   Installed StatisticalTraits ──────────── v3.4.0
   Installed StatsFuns ──────────────────── v1.3.2
   Installed LoggingExtras ──────────────── v1.1.0
   Installed FFTW ───────────────────────── v1.8.0
   Installed RelocatableFolders ─────────── v1.0.1
   In

# 3. Bootstrapping and Decision Trees

3.1 Generar el dataset, transformar las variables categóricas en dummies.

In [ ]:
println("Generamos el dataset base:")
using RDatasets, DataFrames
Hitters = dataset("ISLR", "Hitters")
Hitters[1:5, :]

Generamos el dataset:


Row,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
,Int32,Int32,Int32,Int32,Int32,Int32,Int32,Int32,Int32,Int32,Int32,Int32,Int32,Cat…,Cat…,Int32,Int32,Int32,Float64?,Cat…
1,293,66,1,30,29,14,1,293,66,1,30,29,14,A,E,446,33,20,missing,A
2,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.0,N
3,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.0,A
4,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,500.0,N
5,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,91.5,N


In [49]:
Hitters

Row,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
,Int32,Int32,Int32,Int32,Int32,Int32,Int32,Int32,Int32,Int32,Int32,Int32,Int32,Cat…,Cat…,Int32,Int32,Int32,Float64,Cat…
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.0,N
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.0,A
3,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,500.0,N
4,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,91.5,N
5,594,169,4,74,51,35,11,4408,1133,19,501,336,194,A,W,282,421,25,750.0,A
6,185,37,1,23,8,21,2,214,42,1,30,9,24,N,E,76,127,7,70.0,A
7,298,73,0,24,24,7,3,509,108,0,41,37,12,A,W,121,283,9,100.0,A
8,323,81,6,26,32,8,2,341,86,6,32,34,8,N,W,143,290,19,75.0,N
9,401,92,17,49,66,65,13,5206,1332,253,784,890,866,A,E,0,0,0,1100.0,A


In [ ]:
println("Identificamos los valores faltantes:")
describe(Hitters, :nmissing)

Identificamos los valores faltantes


Row,variable,nmissing
,Symbol,Int64
1,AtBat,0
2,Hits,0
3,HmRun,0
4,Runs,0
5,RBI,0
6,Walks,0
7,Years,0
8,CAtBat,0
9,CHits,0


In [22]:
println("Habiendo identificado que solo hay valores faltantes en la columna 'Salario', nos deshacemos de estos")
Hitters = dropmissing(Hitters, :Salary);

Habiendo identificado que solo hay valores faltantes en la columna 'Salario', nos deshacemos de estos


In [23]:
println("Definimos nuestro vector y la matriz objetivos")
using MLJ
Y, X = unpack(Hitters, ==(:Salary), !=(:Salary));


Definimos nuestro vector y la matriz objetivos


In [ ]:
println("Verificamos los tipos de datos")
schema(X)

┌───────────┬───────────────┬─────────────────────────────────┐
│ names     │ scitypes      │ types                           │
├───────────┼───────────────┼─────────────────────────────────┤
│ AtBat     │ Count         │ Int32                           │
│ Hits      │ Count         │ Int32                           │
│ HmRun     │ Count         │ Int32                           │
│ Runs      │ Count         │ Int32                           │
│ RBI       │ Count         │ Int32                           │
│ Walks     │ Count         │ Int32                           │
│ Years     │ Count         │ Int32                           │
│ CAtBat    │ Count         │ Int32                           │
│ CHits     │ Count         │ Int32                           │
│ CHmRun    │ Count         │ Int32                           │
│ CRuns     │ Count         │ Int32                           │
│ CRBI      │ Count         │ Int32                           │
│ CWalks    │ Count         │ Int32     

In [ ]:
println("Tranformamos los datos 'Multiclass' en dummies continuas y los datos 'Count' en continuos")
Onehotencoder = @load OneHotEncoder pkg=MLJModels verbosity=0

ohe = Onehotencoder(features = [:League, :Division, :NewLeague])
ohe_machine = machine(ohe, X)
fit!(ohe_machine);
X = MLJ.transform(ohe_machine, X);
coerce!(X, Count => Continuous);

schema(X)

Tranformamos los datos 'Multiclass' en dummies y los 'Count' en continuos


[ Info: Training machine(OneHotEncoder(features = [:League, :Division, :NewLeague], …), …).


┌──────────────┬────────────┬─────────┐
│ names        │ scitypes   │ types   │
├──────────────┼────────────┼─────────┤
│ AtBat        │ Continuous │ Float64 │
│ Hits         │ Continuous │ Float64 │
│ HmRun        │ Continuous │ Float64 │
│ Runs         │ Continuous │ Float64 │
│ RBI          │ Continuous │ Float64 │
│ Walks        │ Continuous │ Float64 │
│ Years        │ Continuous │ Float64 │
│ CAtBat       │ Continuous │ Float64 │
│ CHits        │ Continuous │ Float64 │
│ CHmRun       │ Continuous │ Float64 │
│ CRuns        │ Continuous │ Float64 │
│ CRBI         │ Continuous │ Float64 │
│ CWalks       │ Continuous │ Float64 │
│ League__A    │ Continuous │ Float64 │
│ League__N    │ Continuous │ Float64 │
│ Division__E  │ Continuous │ Float64 │
│ Division__W  │ Continuous │ Float64 │
│ PutOuts      │ Continuous │ Float64 │
│ Assists      │ Continuous │ Float64 │
│ Errors       │ Continuous │ Float64 │
│ NewLeague__A │ Continuous │ Float64 │
│ NewLeague__N │ Continuous │ Float64 │


3.2 Dividir el sample en dos sets: Training (90%) y Test (10%)

In [33]:
train, test = partition(eachindex(Y), 0.9, rng = 1);

3.3 Haz una regresión OSL y predice el salario de los Hitters usando todas las características del dataset y proporciona intervalos de confianza bootstrap

In [34]:
DecisionTreeRegressor = (@load DecisionTreeRegressor pkg=DecisionTree verbosity=0)
tree_model = DecisionTreeRegressor()
tree_machine = machine(tree_model, X[train, :], Y[train])
fit!(tree_machine);

[ Info: Training machine(DecisionTreeRegressor(max_depth = -1, …), …).


In [59]:
predictions = predict(tree_machine, X[test, :])
sqrt(mean((predictions - Y[test]) .^ 2))

212.79287325701367

In [121]:
using Statistics

N = nrow(Hitters)
Boot_reps = 10000
Y_hat_bootstrap = zeros(Boot_reps)
Y_rand = rand(N)

for i in 1:Boot_reps
    Y_bootstrap = rand(Y_rand, Boot_reps)
    Y_hat_bootstrap[i] = mean(Y_bootstrap)
end

Y_boot = rand(Y_rand, Boot_reps)
Y_hat_boot = mean(Y_boot)

0.537836148588902

In [118]:
lower_bound_boost = quantile(Y_hat_bootstrap, 0.025)

0.5247063016056147

In [119]:
upper_bound_boost = quantile(Y_hat_bootstrap, 0.975)

0.5363144555904092